# Intializing the data given in the question


In [1]:
#Inputing the given data
header=["Age","Salary","Class"]
training_data=[ [30,65,"G"],
                [23,15,"B"],
                [40,75,"G"],
                [55,40,"B"],
                [55,100,"G"],
                [45,60,"G"],
                ]

# Defining basic functions

In [2]:
#functions to calculate unique values for a column in a given dataset
def unique_vals(rows,col):
    return set([row[col] for row in rows])

In [3]:
# a dictionary with the classes in a dataframe and the number of each class.
def class_counts(rows): #returns a dictionary with the class and the count for that class
    counts={} #empty dictionary with count for every class
    for row in rows:
        label=row[-1]
        if label not in counts:
            counts[label]=0
        counts[label]+=1
    return counts

# Asking meaningful questions for partitioning the dataset

In [4]:
#A class to create ask a meaningful question for a given attribute. This question is used to partition the dataset
#It takes the index of the column in the dataframe and also the value which needs to be checked for.
#In this model, since the input test data is not huge, I have not pre=sorted the array as the subsequent code is effective irrespective of whether it is sorted or not.
#Therefore, I have also not taken the average of the value v[i],v[i+1] with the minimum gini index at value v[i]. I have taken value v[i] for simplicity of the code.
class Question:
    #initialising the variables
    def __init__(self,col_index,value):
        self.col_index=col_index
        self.value=value

    #'match' method is used to compare the feature value in an example 
    # to the feature value stored in the question.
    def match(self,example):
        val=example[self.col_index]
        return val<=self.value
    
    # This is just a helper method to print
    # the question in a readable format.
    def __repr__(self):
        a=header[self.col_index]
        b=str(self.value)
        return f"Is {a} <= {b}?"

#Question(1,3)
#q=Question(0,35)
#example=training_data[0]
#q.match(example)

# Partitioning the rows into true and false branches

In [5]:
#Partitions the rows into true and false/ left and right rows, with the question given to the function.
def partition(rows,question):
    true_rows,false_rows=[],[]
    for example in rows:
        if question.match(example):
            true_rows.append(example)
        else:
            false_rows.append(example)
    return true_rows,false_rows
#true_rows, false_rows = partition(training_data, Question(0, 35))
#true_rows

# Calculating the Gini impurity and information gain

In [6]:
#Calculates the gini impurity/gini index of the input dataframe/ list of rows.
def gini_index(rows):
    counts=class_counts(rows)
    impurity=1
    for label in counts:
        prob_label=counts[label]/len(rows)
        impurity-=(prob_label)**2
    return impurity

#calculates the info gain/ cumulative gini index/impurity.
def info_gain(left,right,current_uncertainty):
    len1=len(left)
    len2=len(right)
    p=float(len1/(len1+len2))
    return current_uncertainty-(p*(gini_index(left))+(1-p)*(gini_index(right)))

# Finding the best split

In [7]:
#function to find the best split from the given dataframe by iterating over 
# each feature/ value and calculating the cumulative gini impurity.
def find_best_split(rows):
    best_gain=0 # keep track of the best information gain
    best_question=None # keep track of the feature / value that produced it
    current_uncertainty=gini_index(rows)
    n_features=len(rows[0])-1 #no of columns

    # running the iterations over all the features
    for col in range(n_features): # for each feature
        values=set([row[col] for row in rows]) # unique values in the column

        for val in values: # for each value
            question=Question(col,val)
            true_rows,false_rows=partition(rows,question) # try splitting the dataset
            if len(true_rows)==0 or len(false_rows)==0: # Skip this split if it doesn't divide the dataset.
                continue
            gain=info_gain(true_rows,false_rows,current_uncertainty) # Calculate the information gain from this split
            # updating the values if the comparison condition is satisfied
            if gain>=best_gain:
                best_gain,best_question=gain,question
    return best_gain,best_question

# Identifying a leaf node and a decision node

In [8]:
# A Leaf node classifies data.
# References to the number of times a class appears in the rows 
# from the training data that reach this leaf.
class Leaf:
    def __init__(self,rows):
        self.predictions=class_counts(rows)
# A Decision Node asks a question to find out the split.
# references to the question, and to the two child nodes.
class Decision_Node:
    def __init__(self,question,true_branch,false_branch):
        self.question=question
        self.true_branch=true_branch
        self.false_branch=false_branch

# Building the tree

In [9]:
#the main function where the tree is built. It is a recursive function that returns the Decision Node 
# if gain is not equal to zero and a leaf if the gain is zero.
def build_tree(rows):
    if len(rows)==len(training_data):
        # intializing the root partition as given in the question
        question=Question(0,35)
        true_rows,false_rows=partition(rows,question)
        uncertainty=gini_index(rows)
        gain=info_gain(true_rows,false_rows,uncertainty)
    else:
        # else find the best split for the subsequent nodes
        gain, question = find_best_split(rows)
        
        # Base case: no further info gain
        # Since we can ask no further questions (since we have no more partitions to make),
        # we'll return a leaf.
        if gain==0:
            return Leaf(rows)
        
        # If we reach here, we have found a useful feature / value
        # to partition on.
        true_rows, false_rows=partition(rows,question)
    
    # build the true branch.    
    true_branch=build_tree(true_rows)
    # build the false branch.
    false_branch=build_tree(false_rows)

    # Return a Question node.
    # This records the best feature / value to ask at this point,
    # as well as the branches to follow
    # depending on the answer.
    return Decision_Node(question,true_branch,false_branch)

# Printing the tree

In [10]:
#funtcion used to print the tree.
def print_tree(node, spacing=""):

    # Base case: we've reached a leaf
    if isinstance(node, Leaf):
        print (spacing + "Predict", node.predictions)
        return

    # Print the question at this node
    print (spacing + str(node.question))

    # Call this function on the true branch
    print (spacing + '--> True:')
    print_tree(node.true_branch, spacing + "  ")

    # Call this function on the false branch
    print (spacing + '--> False:')
    print_tree(node.false_branch, spacing + "  ")

# Output

In [11]:
#Inputing the training data and printing out the results
my_tree=build_tree(training_data)
print_tree(my_tree)

Is Age <= 35?
--> True:
  Is Salary <= 15?
  --> True:
    Predict {'B': 1}
  --> False:
    Predict {'G': 1}
--> False:
  Is Salary <= 40?
  --> True:
    Predict {'B': 1}
  --> False:
    Predict {'G': 3}


# References


1.   Professor Sundar's Class Notes
2.   SLIQ Model Paper | IBM
3.   [Decision and Classification Trees | StatQuest with Josh Starmer](https://youtu.be/_L39rN6gz7Y)
4.   [Let’s Write a Decision Tree Classifier from Scratch - Machine Learning Recipes | Google Developers](https://youtu.be/LDRbO9a6XPU)


